In [ ]:
!pip install transformers 

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Для наглядности будем работать с русскоязычной GPT от Сбера.
# Ниже команды для загрузки и инициализации модели и токенизатора.
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import math
import pandas as pd

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
gpt3_large = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [ ]:
import torch
# Just right before the actual usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
gpt3_large = gpt3_large.to(device)

In [ ]:
def calculate_perplexity(sentence, model, tokenizer):
    sentence_positive = 'довольна:'+sentence               
    sentence_negative = 'недовольна:'+sentence               
    list_sent = [sentence_positive, sentence_negative]
    ppl_values = []

    for sentence in list_sent:
      encodings = tokenizer(sentence, return_tensors='pt')
      input_ids = encodings.input_ids.to(device)
      with torch.no_grad():
          outputs = gpt3_large(input_ids=input_ids, labels=input_ids)
      loss = outputs.loss
      ppl = math.exp(loss.item() * input_ids.size(1))
      ppl_values.append(ppl)
    
    if ppl_values[0] > ppl_values[1]:
      return 'отрицательный'
    elif ppl_values[0] < ppl_values[1]:
      return 'положительный'

In [ ]:
reviews = pd.read_excel('test.xlsx')
reviews = reviews.dropna()

In [ ]:
reviews['sentiment'] = reviews['mark_6'].apply(lambda x: calculate_perplexity(x, model, tokenizer))

In [ ]:
reviews

In [ ]:
calculate_perplexity("все хорошо", model, tokenizer)

In [ ]:
reviews['sentiment'] = reviews.apply(lambda x: calculate_perplexity(reviews['mark_6'], model, tokenizer))

In [ ]:
reviews.dtype()